## Energy saved from recycling
<p>Did you know that recycling saves energy by reducing or eliminating the need to make materials from scratch? For example, aluminum can manufacturers can skip the energy-costly process of producing aluminum from ore by cleaning and melting recycled cans. Aluminum is classified as a non-ferrous metal.</p>
<p>Singapore has an ambitious goal of becoming a zero-waste nation. The amount of waste disposed of in Singapore has increased seven-fold over the last 40 years. At this rate, Semakau Landfill, Singapore’s only landfill, will run out of space by 2035. Making matters worse, Singapore has limited land for building new incineration plants or landfills.</p>
<p>The government would like to motivate citizens by sharing the total energy that the combined recycling efforts have saved every year. They have asked you to help them.</p>
<p>You have been provided with three datasets. The data come from different teams, so the names of waste types may differ.</p>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
    <div style="font-size:16px"><b>datasets/wastestats.csv - Recycling statistics per waste type for the period 2003 to 2017</b>
    </div>
    <div>Source: <a href="https://www.nea.gov.sg/our-services/waste-management/waste-statistics-and-overall-recycling">Singapore National Environment Agency</a></div>
<ul>
    <li><b>waste_type: </b>The type of waste recycled.</li>
    <li><b>waste_disposed_of_tonne: </b>The amount of waste that could not be recycled (in metric tonnes).</li>
    <li><b>total_waste_recycle_tonne: </b>The amount of waste that could be recycled (in metric tonnes).</li>
    <li><b>total_waste_generated: </b>The total amount of waste collected before recycling (in metric tonnes).</li>
    <li><b>recycling_rate: </b>The amount of waste recycled per tonne of waste generated.</li>
    <li><b>year: </b>The recycling year.</li>
</ul>
    </div>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6; margin-top: 17px;">
    <div style="font-size:16px"><b>datasets/2018_2019_waste.csv - Recycling statistics per waste type for the period 2018 to 2019</b>
    </div>
    <div> Source: <a href="https://www.nea.gov.sg/our-services/waste-management/waste-statistics-and-overall-recycling">Singapore National Environment Agency</a></div>
<ul>
    <li><b>Waste Type: </b>The type of waste recycled.</li>
    <li><b>Total Generated: </b>The total amount of waste collected before recycling (in thousands of metric tonnes).</li> 
    <li><b>Total Recycled: </b>The amount of waste that could be recycled. (in thousands of metric tonnes).</li>
    <li><b>Year: </b>The recycling year.</li>
</ul>
    </div>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6; margin-top: 17px;">
    <div style="font-size:16px"><b>datasets/energy_saved.csv -  Estimations of the amount of energy saved per waste type in kWh</b>
    </div>
<ul>
    <li><b>material: </b>The type of waste recycled.</li>
    <li><b>energy_saved: </b>An estimate of the energy saved (in kiloWatt hour) by recycling a metric tonne of waste.</li> 
    <li><b>crude_oil_saved: </b>An estimate of the number of barrels of oil saved by recycling a metric tonne of waste.</li>
</ul>

</div>
<pre><code>
</code></pre>

In [2]:
import pandas as pd

wastestats = pd.read_csv("datasets/wastestats.csv")
wastestats_filter = wastestats[wastestats['waste_type']
    .isin(['Ferrous metal', 'Non-ferrous metal', 'Ferrous Metal', 
    'Non-ferrous Metals', 'Ferrous Metals', 'Non-ferrous metals', 
    'Glass', 'Plastics', 'Plastic'])]
wastestats_filter = wastestats_filter[wastestats_filter['year'] > 2014].reset_index()

_2018_2019_waste = pd.read_csv("datasets/2018_2019_waste.csv")
_2018_2019_waste_filter = _2018_2019_waste[_2018_2019_waste['Waste Type']
    .isin(['Ferrous Metal', 'Glass', 'Plastics', 'Non-Ferrous Metal'])].reset_index()
_2018_2019_waste_filter = _2018_2019_waste_filter.rename(columns={'Year': 'year','Waste Type': 'waste_type',"Total Recycled ('000 tonnes)": 'total_waste_recycled_tonne'})
_2018_2019_waste_filter['total_waste_recycled_tonne'] = _2018_2019_waste_filter['total_waste_recycled_tonne'] * 1000

energy_saved = pd.read_csv("datasets/energy_saved.csv")
energy_saved.columns = energy_saved.iloc[2]
energy_saved_data = energy_saved.iloc[3:4,1:5].copy()
energy_saved_data = energy_saved_data.apply(lambda x: x.str.replace(' Kwh', ''))
energy_saved_data = energy_saved_data.apply(pd.to_numeric)
energy_saved_data['Plastics'] = energy_saved_data['Plastic']
energy_saved_data['Ferrous metal'] = energy_saved_data['Ferrous Metal']
energy_saved_data['Ferrous Metals'] = energy_saved_data['Ferrous Metal']
energy_saved_data['Non-ferrous metal'] = energy_saved_data['Non-Ferrous Metal']
energy_saved_data['Non-ferrous metals'] = energy_saved_data['Non-Ferrous Metal']
energy_saved_data_new = energy_saved_data.T.reset_index()
energy_saved_data_new = energy_saved_data_new.rename(columns={2: 'waste_type', 3: 'energy_saved'})
energy_saved_data_new.set_index('waste_type',inplace=True)

wastest = pd.merge(wastestats_filter, _2018_2019_waste_filter, how='outer')
wastest = pd.merge(wastest, energy_saved_data_new, how='outer',left_on='waste_type', right_index=True)
wastest = wastest.sort_values('year').reset_index(drop = True)
wastest.set_index('year',inplace=True)
wastest['total_energy_saved'] = wastest['total_waste_recycled_tonne'] * wastest['energy_saved']

annual_energy_savings = wastest.groupby(['year'])['total_energy_saved'].sum().reset_index()
annual_energy_savings.set_index('year',inplace=True)

#wastestats['waste_type'].unique()
#wastestats
#wastestats_filter
#_2018_2019_waste
#_2018_2019_waste_filter
#energy_saved
#energy_saved_data
#energy_saved_data_new
#wastest
#annual_energy_savings